<a href="https://colab.research.google.com/github/ShinAsakawa/2019seminar_info/blob/master/notebooks/2019si_miniXOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h1>[Python で 超実習ディープラーニング](https://www.seminar-info.jp/entry/seminars/view/1/4174)</h1>
<h3><strong>実践！ 強化学習・画像認識・自然言語処理・ロボティクス</strong></h3>
</center>

<center>
![](https://www.seminar-info.jp/entry/img/logo_ov.jpg)
</center>

<div align='right'>
<a href='mailto:asakawa@ieee.org'>Shin Aasakawa</a>, all rights reserved.<br>
Date: 15/Mar/2019<br>
 MIT license
</div>

In [0]:
import numpy as np                                 # numpy ライブラリの輸入
np.set_printoptions(precision=2)                   # 表示桁数の設定
np.random.seed(109)                                 # 乱数系列の初期化
X = np.array([ [0,0,1],[0,1,1],[1,0,1],[1,1,1] ])
y = np.array([[0,1,1,0]]).T
w0 = 2 * np.random.random((3,4)) - 1
w1 = 2 * np.random.random((4,1)) - 1
for i in range(500):
    l1 = np.tanh(np.dot(X,w0))                     # tanh
    #l1 = 1/(1+np.exp(-(np.dot(X,w0))))            # sigmoid
    l2 = 1/(1+np.exp(-(np.dot(l1,w1))))
    dl2 = (y - l2) * (l2 * (1 - l2))
    dl1 = dl2.dot(w1.T) * (1 - l1 ** 2)            # tanh
    #dl1 = dl2.dot(w1.T) * (11 * (1-l1))           # sigmoid
    w1 += np.dot(l1.T, dl2)
    w0 += np.dot(X.T, dl1)
    print(l2.T) if i % 100 == 0 else None
    #if np.sum(np.square(y - l2)) < 0.01:
    #    print(i, l2.T)
    #    break

[[0.26 0.61 0.22 0.54]]
[[0.06 0.9  0.89 0.1 ]]
[[0.03 0.95 0.94 0.05]]
[[0.02 0.96 0.96 0.04]]
[[0.02 0.97 0.97 0.03]]


In [0]:
# 上の例は中間層から出力層へ行くバイアス項が無いので修正
import numpy as np
np.set_printoptions(precision=2)
np.random.seed()
n_hid = 10
X = np.array([ [0,0],[0,1],[1,0],[1,1]])
y = np.array([[0,1,1,0]]).T
b = np.ones((X.shape[0],1))
w1 = 2 * np.random.random((X.shape[1]+1, n_hid)) - 1
w2 = 2 * np.random.random((n_hid+1,y.shape[1])) - 1
iter_max = 500
for i in range(iter_max):
    l1 = 1/(1+np.exp(-(np.dot(np.concatenate((X,b),axis=1),w1))))  # sigmoid
    #l1 = np.tanh(np.dot(np.concatenate((X,b),axis=1),w1))           # tanh
    l2 = 1/(1+np.exp(-(np.dot(np.concatenate((l1,b),axis=1),w2))))
    #dl2 = (y - l2) * (l2 * (1 - l2))                               # sigmoid
    dl2 = (y - l2) * (l2 - l2 **2)                                  # tanh
    dl1 = np.dot(dl2, w2.T) * (np.concatenate((l1,b),axis=1) * (1-np.concatenate((l1,b),axis=1)))
    w2 += np.dot(np.concatenate((l1,b),axis=1).T, dl2)
    w1 += np.dot(np.concatenate((X,b),axis=1).T, dl1[:,:-1])
    print(l2.T) if i % 100 == 0 else None
#print(l2.T)

[[0.86 0.84 0.84 0.82]]
[[0.45 0.52 0.55 0.49]]
[[0.22 0.7  0.73 0.34]]
[[0.11 0.86 0.87 0.16]]
[[0.07 0.91 0.91 0.1 ]]


----

他の手法を使うとどうなるか？

## ロジスティック回帰の場合

In [0]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X,y)
model.score(X,y)
model.predict(X)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 0, 0])

---

## ランダムフォレストの場合


In [0]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X,y)
model.score(X,y)
model.predict(X)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


array([0, 1, 1, 0])

---

## SVM の場合


In [0]:
from sklearn.svm import SVC
model = SVC(kernel='rbf')
model.fit(X,y)
model.score(X,y)
model.predict(X)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


array([0, 1, 1, 0])

---

### SVM の場合(2)

In [0]:
model = SVC(kernel='sigmoid')
model.fit(X,y)
model.score(X,y)
model.predict(X)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


array([1, 1, 1, 1])